# Predicting FIFA World Cup 2022

## Importing libraries

In [186]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

## Reading the database

In [187]:
final_df = pd.read_csv('data/train_features.csv')
final_df.tail()

,home_team,away_team,home_team_fifa_rank,away_team_fifa_rank,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score,home_team_result_numeric
9316,Netherlands,Wales,10,18,81.0,74.0,85.0,83.0,84.0,75.0,73.0,78.0,1
9317,Poland,Belgium,26,2,87.0,89.0,75.0,85.0,76.0,81.0,86.0,86.0,0
9318,Chile,Ghana,28,60,79.0,74.0,76.0,77.0,78.0,76.0,76.0,78.0,0
9319,Japan,Tunisia,23,35,73.0,64.0,75.0,75.0,78.0,71.0,72.0,74.0,0
9320,Korea Republic,Egypt,29,32,75.0,70.0,73.0,80.0,74.0,70.0,79.0,71.0,1


### Train data 

get X and y

In [188]:
y=final_df['home_team_result_numeric']
#y=pd.get_dummies(final_df['home_team_result_numeric'])
X=final_df.iloc[:,2:-1]

scaling

In [189]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X)

slit data to train and test 

In [190]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
X_hold_test, X_test, y_hold_test, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

metrics function

In [191]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix

In [192]:
def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict,average='macro')
    print("f1: %.2f%%" % (f1 * 100.0)) 
    rec = recall_score(y_test, y_predict, average='macro')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='macro')
    print("precision: %.2f%%" % (prc * 100.0)) 

In [193]:
from sklearn.metrics import classification_report,ConfusionMatrixDisplay
def metrics_display(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(classification_report(y_test,y_pred))
    ConfusionMatrixDisplay.from_predictions(y_test,y_pred);

### Train the model 

Neural network


In [194]:
import tensorflow.keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras import Input

X_train.shape

(8388, 10)

In [195]:
model = Sequential()
model.add(Input(shape=(10,)))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(3,activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train,epochs=10,validation_split=0.2)

Epoch 1/10


Exception ignored in: <function EagerResourceDeleter.__del__ at 0x000002258C034DC0>
Traceback (most recent call last):
  File "c:\Users\Bruno Dutra\anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py", line 289, in __del__
    gen_resource_variable_ops.destroy_resource_op(
  File "c:\Users\Bruno Dutra\anaconda3\lib\site-packages\tensorflow\python\ops\gen_resource_variable_ops.py", line 242, in destroy_resource_op
    _result = pywrap_tfe.TFE_Py_FastPathExecute(
KeyboardInterrupt: 


210/210 [==============================] - 7s 6ms/step - loss: 17.4494 - accuracy: 0.3391 - val_loss: 0.8274 - val_accuracy: 0.4565
Epoch 2/10
210/210 [==============================] - 1s 5ms/step - loss: 0.8524 - accuracy: 0.3257 - val_loss: 0.6217 - val_accuracy: 0.5066
Epoch 3/10
210/210 [==============================] - 1s 6ms/step - loss: 0.6751 - accuracy: 0.3397 - val_loss: 0.6291 - val_accuracy: 0.2896
Epoch 4/10
210/210 [==============================] - 1s 5ms/step - loss: 0.6480 - accuracy: 0.3247 - val_loss: 0.5606 - val_accuracy: 0.5107
Epoch 5/10
210/210 [==============================] - 1s 4ms/step - loss: 0.5706 - accuracy: 0.3343 - val_loss: 0.5529 - val_accuracy: 0.2920
Epoch 6/10
210/210 [==============================] - 1s 4ms/step - loss: 0.5627 - accuracy: 0.3441 - val_loss: 0.5628 - val_accuracy: 0.2920
Epoch 7/10
210/210 [==============================] - 1s 5ms/step - loss: 0.5486 - accuracy: 0.3600 - val_loss: 0.5233 - val_accuracy: 0.3135
Epoch 8/10
210/2

In [ ]:

y_pred1 = model.predict(X_test)
y_pred1 = np.argmax(y_pred1,axis=1)

print("confusion_matrix: ")
evaluate_results(y_test, y_pred1)
print(classification_report(y_test,y_pred1))
ConfusionMatrixDisplay.from_predictions(y_test,y_pred1);

In [ ]:
from sklearn import svm, metrics
from sklearn.metrics import confusion_matrix
#%% Using SVM model to test 
#=============================================================================
clf = svm.SVC(kernel='rbf',gamma='auto',C=1E6)
clf.fit(X_train,y_train)

y_pred2=clf.predict(X_test)

print("Classification report for classifier %s:\n%s\n"
      % (clf, metrics.classification_report(y_test, y_pred2)))

# Plot non-normalized confusion matrix
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
cnf_matrix = confusion_matrix(y_test, y_pred2)



y_pred=clf.predict(X)
fig, axs = plt.subplots(2, 1)
fig.suptitle('Predicted moviments')
axs[0].plot( y, 'b')
axs[1].plot( y_pred2, 'r')
plt.show()

ValueError: y should be a 1d array, got an array of shape (8388, 3) instead.